In [ ]:
# import main libraries/packages
import warnings
warnings.filterwarnings('ignore') # to ignore annoying IPython warnings
import numpy as np
import pandas as pd
import gzip
import re

from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix, hstack # to get memory-efficient representation of matrices (sparse format)
from textblob import TextBlob, Word

# preprocessing / feature extraction / feature transformation
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer

# models
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline, FeatureUnion

# metrics/validation
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
# model serialization/deserialization
import dill

### Unpacking from .json

In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)
        
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

'''
df = getDF('reviews_Movies_and_TV_5.json.gz')
train, test = train_test_split(df.asin.unique(), test_size=0.1, random_state=42)
df_train = df[df.asin.isin(train) & (df.overall != 3)].copy()
df_test = df[df.asin.isin(test) & (df.overall != 3)].copy()
df_train['overall'] = df_train['overall'].apply(lambda x: 1 if x > 3 else 0)
df_test['overall'] = df_test['overall'].apply(lambda x: 1 if x > 3 else 0)
df_train[['overall', 'reviewText', 'asin']].to_csv('csv/train.csv', index=False)
df_test[['overall', 'reviewText']].to_csv('csv/test.csv', index=False)
'''

pass

# loading datasets

In [ ]:
# load train dataset
train_df_names = ['csv/train.csv']

df = pd.concat((pd.read_csv(name, engine='c', sep=',', 
                 usecols=['overall', 'reviewText']) for name in train_df_names), ignore_index=True)

df = df.rename(columns={'overall' : 'label', 'reviewText' : 'text'}) # rename columns

# load test dataset
df_test = pd.read_csv('csv/test.csv', sep=",", engine='c', usecols=['overall', 'reviewText'])
df_test = df_test.rename(columns={'overall' : 'label', 'reviewText' : 'text'})

# other test datasets
test_RT = pd.read_csv("csv/reviews_rt_all.csv", sep="|", engine='c', usecols=['label', 'text'])
test_imdb = pd.read_csv("csv/imdb_small.csv", sep="|", engine='c', usecols=['label', 'text'])
test_polarity_RT = pd.read_csv("csv/test_reviews.csv", sep="|", engine='c', usecols=['label', 'text'])

# to avoid some errors in 'object'-typed column
df.text = df.text.apply(str)
df_test.text = df_test.text.apply(str)
test_RT.text = test_RT.text.apply(str)
test_imdb.text = test_imdb.text.apply(str)
test_polarity_RT.text = test_polarity_RT.text.apply(str)

print('review count: {}'.format(len(df)))

In [ ]:
df.head()

In [ ]:
%%time
# check for class balance
print('class balance Train set:', '\n', df.label.value_counts())
print('\n','class balance Test set:', '\n', df_test.label.value_counts())
print('\n','class balance Test RT set:', '\n', test_RT.label.value_counts())
print('\n','class balance Test imdb set:', '\n', test_imdb.label.value_counts())
print('\n','class balance Test polarity RT set:', '\n', test_polarity_RT.label.value_counts())

In [ ]:
%%time
# imbalanced data
#df_small = df.sample(500000, random_state=50)

# balanced data
#df_small = pd.concat([df[df.label == 1].sample(185410, random_state=42),  
#                      df[df.label == 0].sample(185410, random_state=42)])

df_small = df.sample(1343971, random_state=42)


print('class balance:', '\n', df_small.label.value_counts())

In [ ]:
df_small.head()

# Preprocessing

In [ ]:
%%time
def get_rate(s):
    candidates = re.findall(r'(\d{1,3}[\\|/]{1}\d{1,2})', s)
    rates = []
    for c in candidates:
        try:
            rates.append(eval(c))
        except SyntaxError:
            pass
        except ZeroDivisionError:
            return 0
    return np.median(rates)

# regular expression to split review on sentences
sentence_splitter = re.compile('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<![A-Z]\.)(?<=\!|\?|\.)\s')

# lists of positive/negative smiles
positive_smiles = set([
":‑)",":)",":-]",":]",":-3",":3",":->",":>","8-)","8)",":-}",":}",":o)",":c)",":^)","=]","=)",":‑D",":D","8‑D","8D",
"x‑D","xD","X‑D","XD","=D","=3","B^D",":-))",";‑)",";)","*-)","*)",";‑]",";]",";^)",":‑,",";D",":‑P",":P","X‑P","XP",
"x‑p","xp",":‑p",":p",":‑Þ",":Þ",":‑þ",":þ",":‑b",":b","d:","=p",">:P", ":'‑)", ":')",  ":-*", ":*", ":×"
])
negative_smiles = set([
":‑(",":(",":‑c",":c",":‑<",":<",":‑[",":[",":-||",">:[",":{",":@",">:(","D‑':","D:<","D:","D8","D;","D=","DX",":‑/",
":/",":‑.",'>:\\', ">:/", ":\\", "=/" ,"=\\", ":L", "=L",":S",":‑|",":|","|‑O","<:‑|"
])

# pattern to catch SUCH WORDS and ignore SuCH :)
uppercase_pattern = re.compile(r'(\b[0-9]*[A-Z]+[0-9]*[A-Z]{1,}[0-9]*\b)')

# contrast conjugations
contrast_conj = set([
'alternatively','anyway','but','by contrast','differ from','elsewhere','even so','however','in contrast','in fact',
'in other respects','in spite of','in that respect','instead','nevertheless','on the contrary','on the other hand',
'rather','though','whereas','yet'])

# to get review "purity" ~ same sentiment over review (~1) or not (~0)
def purity(sentences):
    polarities = np.array([TextBlob(x).sentiment.polarity for x in sentences])
    return polarities.sum() / np.abs(polarities).sum()

# feature engineering ^-^
def get_custom_features(text):
    # assume text = pd.Series with review text
    print('extracting custom features...')
    tdf = pd.DataFrame()
    tdf['text'] = text 
    tdf['sentences'] = tdf.text.apply(lambda s: re.split(sentence_splitter, s)) # split to sentences 
    
    # feature 4 - totally uppercase words (like HOLY JESUS!)
    tdf['upper_word_cnt'] = tdf.text.apply(lambda s: len(re.findall(uppercase_pattern, s)))
    
    # try to extract rating :) like "great film. 9/10" will yield 0.9
    tdf['rating'] = tdf['text'].apply(get_rate).fillna(-1) # feature 5 - rating (if found in review)

    # try to extract smiles and count positive/negative smiles per review (features 6,7)
    tdf['positive_smiles'] = tdf.text.apply(lambda s: len([x for x in s.split() if x in positive_smiles]))
    tdf['negative_smiles'] = tdf.text.apply(lambda s: len([x for x in s.split() if x in negative_smiles]))
    
    # polarities (based on SentiWordNet word polarities)
    tdf['polarity_1st_sent'] = tdf.sentences.apply(lambda s: TextBlob(s[0]).sentiment.polarity)
    tdf['polarity_last_sent'] = tdf.sentences.apply(lambda s: TextBlob(s[-1]).sentiment.polarity)
     
    return csr_matrix(tdf[tdf.columns[2:]].values) # to get sparse format

# model 

In [ ]:
# EXTRACTOR
extraction_list = []

# 1. custom features
extraction_list.append(['custom_features', 
                             FunctionTransformer(func=get_custom_features,
                                                 validate=False,
                                                 accept_sparse=True
                                                )
                            ])
# 2. simple bag-of-words (tf-idf)
extraction_list.append(['tfidf', 
                             TfidfVectorizer(decode_error='ignore',
                                             max_df=0.75, 
                                             min_df=3,
                                             ngram_range=(1, 3), 
                                             max_features=35000, 
                                             stop_words=None
                                            )
                            ])

extractor = FeatureUnion(extraction_list)


# CLASSIFIER
clf = LGBMClassifier(
    colsample_bytree=0.7, 
    learning_rate=0.15, 
    max_depth=-1, 
    min_child_samples=15, 
    n_estimators=200, 
    num_leaves=127, 
    reg_lambda=1, 
    scale_pos_weight=1, 
    subsample_for_bin=5000
    )

# create pipeline, combining steps together                                                                                                                       

model = Pipeline(
    [
        ('feature_extraction', extractor),
        ('clf', clf)
    ])

# training model

In [ ]:
%%time
# train/test split
X_train, X_test, y_train, y_test = train_test_split(
                                                    df_small.text, 
                                                    df_small.label, 
                                                    test_size=0.1, 
                                                    random_state=42, 
                                                    stratify=df_small.label
                                                   )



# fit model
model.fit(X_train, y_train)
print('finally fitted :)')

y_pred = model.predict(X_test)

#check results on validation
print('Accuracy on validation: {}'.format(accuracy_score(y_pred, y_test)))
print(classification_report(y_test, y_pred))

# testing model

In [ ]:
%%time
# test predict RT
X, y = test_RT.text, test_RT.label
y_pred_RT = model.predict(X)
print(classification_report(y, y_pred_RT))
print('accuracy: {}'.format(accuracy_score(y, y_pred_RT)))


In [ ]:
%%time
# test predict imdb
X, y = test_imdb.text, test_imdb.label
y_pred_imdb = model.predict(X)
print(classification_report(y, y_pred_imdb ))
print('accuracy: {}'.format(accuracy_score(y, y_pred_imdb)))

In [ ]:
%%time
# test predict polarity_RT
X, y = test_polarity_RT.text, test_polarity_RT.label
y_pred_pol = model.predict(X)
print(classification_report(y, y_pred_pol))
print('accuracy: {}'.format(accuracy_score(y, y_pred_pol)))

In [ ]:
%%time
#test predict
X, y = df_test.text, df_test.label
y_pred = model.predict(X)
print(classification_report(y, y_pred)) # смотреть на значение f1-score в строке для класса 0
print('accuracy: {}'.format(accuracy_score(y, y_pred)))